## Simultaneous Iteration for Eigenvalue Computation

We want to implement an iterative method to simultaneously calculate the $p \leq n$ dominant eigenvalues of a matrix $A\in \mathbb C^{n\times n}$, which we assume to be Hermitian.
The method relies on the reduced QR decomposition, in the form:

$$ QR = M,$$
where:
- $M\in\mathbb C^{n\times p}$.
- $Q\in \mathbb C^{n\times p}$ and its columns form an orthonormal set in $\mathbb C^n$ with respect to the usual inner product.
- $R\in \mathbb C^{p\times p}$ is an upper triangular matrix with real and positive diagonal entries.

#### Question 1
Provide the reduced QR decomposition of $M$ when $p=1$.

#### Question 2
By writing
$$ Q = \begin{pmatrix} \widetilde Q &\mathbf{q}\end{pmatrix},\qquad R = \begin{pmatrix} \widetilde R & \mathbf{r} \\ 0 & \alpha\end{pmatrix}, \qquad M = \begin{pmatrix} \widetilde{M} & \mathbf m\end{pmatrix}$$
with $\widetilde Q,\widetilde M \in \mathbb C^{n\times (p-1)}$, $\widetilde R\in\mathbb C^{(p-1)\times (p-1)}$, $\mathbf r \in \mathbb C^{p-1}$, $\alpha\in \R_+$, and $\mathbf m\in \mathbb C^n$, provide the QR decomposition of $M$ given the decomposition $\widetilde Q \widetilde R = \widetilde M$ of the matrix formed by the first $(p-1)$ columns of $M$. (Use the properties of the QR decomposition).


#### Question 3
Implement a method to calculate the reduced QR decomposition.

In [ ]:
using LinearAlgebra

@views function myQR(M)
    n,p = size(M)

    @assert p <= n "Error: p > n"
    ## Implement the decomposition here
    Q = zero(M)
    R = zeros(eltype(M),p,p)

    return Q,R
end

In [ ]:
M = randn(ComplexF64,100,50)
M = Complex{BigFloat}.(M)
Q,R = myQR(M)

@show norm(Q'Q-I) # Check that the columns of Q are orthogonal
@show all(isreal.(diag(R))) # Check that the diagonal of R is real
@show all(real.(diag(R)) .>= 0) # and positive
@show norm(Q*R-M) # Check that QR = M up to roundoff error


In [ ]:
@time qr(M) # Implementation of LinearAlgebra
@time myQR(M) # Our implementation

#### Question 4
The simultaneous iteration method is defined, given an initial condition $X_0\in\mathbb C^{n\times p}$, by the recurrence relation:
$$ X_{n+1} R_{n+1} = A X_n,$$
where $X_{n+1} R_{n+1}$ is the reduced QR decomposition of $AX_n \in \mathbb C^{n\times p}$. It can be shown that for a Hermitian matrix $A$ and appropriate initial conditions $X_0$, the matrix $X_n$ essentially converges column by column to the matrix formed by the $p$ associated eigenvectors (see lecture notes for a proof).

Implement a function `myEigen` in Julia that takes as arguments: a matrix `M` (assumed Hermitian), an integer `nev` (the number of eigenvalues to compute), an integer `niter` (the number of iterations), and an initial matrix `X₀`. The function should return a vector of `nev` eigenvalues and a matrix formed by an approximation of the dominant `nev` eigenvectors of `M` in columns.

For matrices with entries that are not `BigFloat`, you can compare the results with those given by `LinearAlgebra.eigen` for debugging purposes.

In [ ]:
n,nev = 10,3

## Seed for the random number generator
using Random
Random.seed!(2023)
##

X = randn(n,n)/sqrt(n)
X₀ = randn(n,nev)/sqrt(n)


function myEigen(M,nev,niter,X₀)
    ## Implement the iteration here
    return λs,us
end

M = BigFloat.(I + X'X)

@time λs,us=myEigen(M,nev,500,X₀)

display(λs)
display(us)
display(M*us - λs' .*us)

#### Question 5

Study numerically the convergence speed of the first 3 eigenvectors towards their limit as a function of the number of iterations of the method. It can be assumed that all eigenvectors have converged after a large number (let's say 10000) of iterations to obtain reference values. The difference between two vectors $u$ and $v$ will be measured in terms of the angle formed by the generated subspaces:
$$ \theta_{u,v} = \arccos\left(\frac{|u^\intercal v|}{\|u\|\|v\|}\right).$$
Compare the error at the $k$-th iteration and the quantities $(\lambda_i/\lambda_{i+1})^{-k},\, i=1,2,3$, on a graph with a logarithmic $y$-scale.

To ensure numerically that $0\leq \frac{|u^\intercal v|}{\|u\|\|v\|}\leq 1$, the `clamp` function can be used.

In [ ]:
@time λs,us=myEigen(M,n,10000,randn(n,n)) # on calcule les valeurs et vecteurs propres "convergés"
display(M*us - λs' .*us)

In [ ]:
nev=3

iters = 1:200
errs = [Float64[] for i=1:nev]

for niter in iters
    λhats,uhats = myEigen(M,nev,niter,X₀)
    ## Calculate errors here
end

using Plots,LaTeXStrings
colors = (:red,:blue,:green)
pl = plot(xlabel="Itération k", ylabel="Erreur essentielle",yaxis=:log,legend=:bottomleft)

for i=1:nev
    plot!(pl,iters,errs[i],label=L"θ_{%$i}",color=colors[i])
    plot!(pl,k->(λs[i]/λs[i+1])^-k,label=L"(λ_{%$i}/λ_{%$(i+1)})^{-k}",linestyle=:dot,color=colors[i])
end

pl

# PageRank Algorithm

PageRank is an algorithm used to assign a score to the vertices of a directed graph. It is employed by numerous search engines to rank search results. In this context, the directed graph encodes the links between pages on the World Wide Web: the vertices of the directed graph represent web pages, and the edges represent connections between pages—there is an edge from page $i$ to page $j$ if page $i$ contains a hyperlink to page $j$.

Consider a directed graph $G(V, E)$ with vertices $V = \{1, \dotsc, n\}$ and edges $E$. The graph can be represented by its adjacency matrix $\mathsf A \in \{0, 1\}^{n \times n}$, where the entries are given by
$$
a_{ij} =
\begin{cases}
    1 & \text{if there is an edge from $i$ to $j$,} \\
    0 & \text{otherwise.}
\end{cases}
$$
The idea of the PageRank algorithm, in its simplest form, is to assign scores $r_i$ to the vertices by solving the following system of equations:
$$ \tag{PageRank}
    \forall i \in V, \qquad
    r_i
    = \sum_{j \in \mathcal N(i)} \frac{r_j}{o_j}.
$$
where $o_j$ is the out-degree of vertex $j$, i.e., the number of edges originating from $j$. Here, the sum applies to the set of nodes in $\mathcal N(i)$, representing the set of incoming neighbors of vertex $i$, i.e., those with an edge pointing towards vertex $i$.

Let $\mathbf r = \begin{pmatrix} r_1 & \dots & r_n \end{pmatrix}^T$. It can be shown that solving the system <a href="#pagerank">(PageRank)</a> is equivalent to solving the following problem:
$$ \tag{PageRank-vector}
    \mathbf r =
    \mathsf A^T
    \begin{pmatrix}
        \frac{1}{o_1} & &  \\
                      & \ddots & \\
                      & & \frac{1}{o_n}
    \end{pmatrix}
    \mathbf r =:  \mathsf A^T \mathsf O^{-1} \mathbf r.
$$
In other words, the problem is to find an eigenvector with eigenvalue 1 of the matrix $\mathsf M = \mathsf A^T \mathsf O^{-1}$. Note that at this stage, we have not proven the existence or uniqueness of a solution to this equation. The question of the uniqueness of a solution is related to the graph's connectivity and will not be addressed here. However, we will demonstrate that a solution to the problem exists.

**Remark.** The matrix $\mathsf O^{-1} \mathsf A$ is the transition matrix of a random walk on the directed graph, where at each step, a move is made towards an outgoing neighbor with equal probability for each of them. Solving <a href="#pagerank">(PageRank-vector)</a> is equivalent to finding a stationary distribution of this random walk.

**Questions:**

1. Notice that $\mathsf M$ is a left stochastic matrix, meaning that the sum of the elements in each column is equal to 1.

2. Prove that the eigenvalues of any matrix $\mathsf B \in \mathbb R^{n \times n}$ coincide with those of $\mathsf B^T$. You can use the fact that $\det(\mathsf B) = \det(\mathsf B^T)$.

3. Using the previous points, show that 1 is an eigenvalue, and $\rho(\mathsf M) = 1$. For the second part, find a subordinate matrix norm such that $\lVert\mathsf M\rVert= 1$. This demonstrates the existence of a solution to <a href="#pagerank">(PageRank-vector)</a> and also proves that 1 is the dominant eigenvalue of $\mathsf M$.

4. Implement the PageRank algorithm without using the `SparseArrays` package to rank the pages of the English-language Wikipedia as it was in 2013 (see instructions below for downloading the dataset). After assigning a score to all pages, print the top 10 ranked pages, which should be as follows:
   ```
   United States, United Kingdom, World War II, Latin, France, Germany, English language, China, Canada, India
   ```

5. Write a function `search(keyword)` to perform a search in the database. Here is an example of what this function could return:
   ```
   julia> search("Newton")
   47-element Vector{String}:
    "Isaac Newton"
    "Newton (unit)"
    "Newton's laws of motion"
    …
   ```

**Additional indications**:

- Use the following code to download the data.

In [ ]:
using LinearAlgebra
import Downloads
import Tar

# URL where data can be downloaded
url = "https://urbain.vaes.uk/static/wikidata.tar"

# Download the data
filename = "wikidata.tar"
isfile(filename) || Downloads.download(url, filename)

# Extract data into directory `wikidata`
dirname = "wikidata"
isdir(dirname) || Tar.extract(filename, dirname)

This dataset contains a subset of publicly available data <a href="https://snap.stanford.edu/data/enwiki-2013.html">here</a>. To limit computation time, only 5% of the highest-rated articles have been retained for this exercise. Once the archive is decompressed, the dataset can be loaded into Julia as follows:

In [ ]:
import CSV
import DataFrames

# Read nodes and edges into data frames
nodes_dataframe = CSV.read("$dirname/names.csv", DataFrames.DataFrame)
edges_dataframe = CSV.read("$dirname/edges.csv", DataFrames.DataFrame)

# Convert data to matrices
nodes = Matrix(nodes_dataframe)
edges = Matrix(edges_dataframe)

# The data structures should be self-explanatory
edges_dataframe

 - You can use either the simplified version of the algorithm given in [PageRank-vector](#pagerank), or the enhanced version with a damping factor described on Wikipedia.

- Since the dominant eigenvalue is known *a priori*, the following stopping criterion can be used:
  $$
      \frac{\lVert\mathsf M \widehat{\mathbf r} - \widehat{\mathbf r}\rVert_1}{\lVert\widehat{\mathbf r}\rVert_1} < 10^{-10}.
  $$
  where $\widehat{\mathbf r}$ is an approximation of the eigenvector corresponding to the dominant eigenvalue.

- All methods for solving [PageRank-vector](#pagerank) require computing matrix-vector products with the matrix $\mathsf M$. Since this matrix is very large, it cannot be stored as a dense matrix of type `Matrix`. To solve the problem, it is necessary to define a sparse matrix structure. You can, for example, use the COO sparse matrix format, which is the simplest. Another option is to use the CSR format, which allows more efficient matrix-vector product calculations. It will be useful, for solving the eigenvalue problem,

  * to define the matrix-vector multiplication method `*(A::my_sparse_matrix, b::Vector{Float64})`.

  * to define a function `my_sparse` that constructs a sparse matrix from vectors `rows`, `cols`, and `vals` in COO format. Obviously, if you choose to use the COO format, this function is trivial.

In [ ]:
# Necessary to overload `*`
import Base: *

# Modify if necessary
struct my_sparse_matrix
    rows::Vector{Int}
    cols::Vector{Int}
    vals::Vector{Float64}
    m::Int
    n::Int
end

function *(A::my_sparse_matrix, b::Vector{Float64})
    # Fill me
end

# Constructor from parameters in COO format (modify if necessary)
function my_sparse(rows, cols, vals, m, n)
    my_sparse_matrix(rows, cols, vals, m, n)
end

# Test the code
m, n = 4, 3
R = [2, 2, 2, 3, 3]
C = [1, 2, 3, 1, 3]
V = [5., 6., 7., 8., 9.]
A = my_sparse(R, C, V, m, n)
b = [1.; 1.; 1.]
@assert A*b == [0.; 18.; 17.; 0.] "Multiplication does not work!"

**Note.** Although it is requested not to use `SparseArrays` in your final code, using this package for testing is encouraged. It can be useful, for example, to implement PageRank first with a sparse matrix constructed with `SparseArrays` before using your own sparse matrix structure. As a reminder, a sparse matrix can be constructed from COO format parameters as follows using `SparseArrays`:

In [ ]:
import SparseArrays
A = SparseArrays.sparse(R, C, V, m, n)
@assert A*b == [0.; 18.; 17.; 0.] "Multiplication does not work!"

You can organize the rest of your code in three cells:

- Construct the matrix $\mathsf M$:

In [ ]:
# Fill me

- Solve the PageRank problem (the `sortperm` function may be useful here) and print the names of the top 10 ranked pages:

In [ ]:
# Fill me

- Define and test the `search` function
(The functions `filter` and `occursin` can be useful here):

In [ ]:
function search(keyword)
    # Fill me
end
search("Newton")